In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import torch

C:\Users\dor_b\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = "./orientation/orientation-tr-train.tsv"
data = pd.read_csv(dataset_path, sep="\t")

# Display basic information
print(data.head())
print(data.info())

        id                           speaker sex  \
0  tr00000  ca2031caa4032c51980160359953d507   M   
1  tr00001  4cee0addb3c69f6866869b180f90d45f   M   
2  tr00002  b3d7f76d74ec268492f8190ca123a6b2   M   
3  tr00003  722efac7138c8197a9d1e97eed3a8b18   M   
4  tr00004  be82a4ade406ec6774a0a2e38f6957e3   M   

                                                text  \
0  Yeni yasama döneminin ülkemiz için, milletimiz...   
1  Sayın Başkan, değerli milletvekilleri; bugün, ...   
2  Sayın Başkanım, öncelikle yüce Meclisin Başkan...   
3  24’üncü Dönem Meclis Başkanlığına seçilmenizde...   
4  24’üncü Yasama Dönemimizin tüm milletvekilleri...   

                                             text_en  label  
0  Mr. President, dear lawmakers, I salute you, a...      1  
1  Mr. President, members of lawmakers, as I spea...      1  
2  Mr. President, I'm here to share with you the ...      1  
3  Mr. President, under the principles determined...      1  
4  Mr. President, dear lawmakers, I ask 

In [3]:
# Drop rows with missing translations or text fields
data = data.dropna(subset=['text_en', 'label'])

# Display class distribution
print(data['label'].value_counts())


label
1    9390
0    6748
Name: count, dtype: int64


In [4]:
train_data, test_data = train_test_split(
    data, test_size=0.1, stratify=data['label'], random_state=42
)
print(f"Training size: {len(train_data)}, Test size: {len(test_data)}")

Training size: 14524, Test size: 1614


In [5]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text_en"], padding="max_length", truncation=True)


C:\Users\dor_b\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns
tokenized_train = tokenized_train.remove_columns(["text_en", "__index_level_0__"])
tokenized_test = tokenized_test.remove_columns(["text_en", "__index_level_0__"])

# Set format for PyTorch
tokenized_train.set_format("torch")
tokenized_test.set_format("torch")


Map: 100%|██████████| 1614/1614 [00:01<00:00, 1132.44 examples/s]


In [10]:

print(tokenized_test)

Dataset({
    features: ['id', 'speaker', 'sex', 'text_en', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1614
})


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
output_dir = 'C:\\Users\\dor_b\\Documents\\results'
log = 'C:\\Users\\dor_b\\Documents\\logs'
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir=log,
    logging_steps=50,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
)
trainer.train()


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.den

{'loss': 0.6514, 'learning_rate': 2.944933920704846e-05, 'epoch': 0.06}


  4%|▎         | 100/2724 [44:14<19:13:49, 26.38s/it]

{'loss': 0.6637, 'learning_rate': 2.889867841409692e-05, 'epoch': 0.11}


  6%|▌         | 150/2724 [1:06:15<18:51:39, 26.38s/it]

...

{'eval_loss': 0.3190705180168152, 'eval_runtime': 481.8603, 'eval_samples_per_second': 3.35, 'eval_steps_per_second': 0.21, 'epoch': 3.0}


100%|██████████| 2724/2724 [20:20:43<00:00, 25.34s/it]

{'train_runtime': 73243.8627, 'train_samples_per_second': 0.595, 'train_steps_per_second': 0.037, 'train_loss': 0.3518685877935827, 'epoch': 3.0}


100%|██████████| 2724/2724 [20:20:44<00:00, 26.89s/it]


TrainOutput(global_step=2724, training_loss=0.3518685877935827, metrics={'train_runtime': 73243.8627, 'train_samples_per_second': 0.595, 'train_steps_per_second': 0.037, 'train_loss': 0.3518685877935827, 'epoch': 3.0})

In [8]:
model.save_pretrained("./xlm_roberta_model")
tokenizer.save_pretrained("./xlm_roberta_model")

('./xlm_roberta_model\\tokenizer_config.json',
 './xlm_roberta_model\\special_tokens_map.json',
 './xlm_roberta_model\\tokenizer.json')

In [9]:
from sklearn.metrics import classification_report

# Get predictions
predictions = trainer.predict(tokenized_test)
preds = np.argmax(predictions.predictions, axis=-1)

# Evaluate
labels = test_data['label'].values
print(classification_report(labels, preds))


100%|██████████| 101/101 [07:58<00:00,  4.74s/it]

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       675
           1       0.91      0.88      0.89       939

    accuracy                           0.88      1614
   macro avg       0.87      0.88      0.88      1614
weighted avg       0.88      0.88      0.88      1614



In [2]:
import transformers
import torch
import huggingface_hub



model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
    use_auth_token="your token here",
    trust_remote_code=True
)

def classification_pipeline(speech):

    messages = [
        {"role": "instruction", "content": """Assume you are politician, which will be asked to classify the ideology of the
                                                speakers' party from turkish parliament. Speeach will be given as a text. 
                                                In other words, this involves performing binary classification to determine whether
                                                the speaker’s party leans left (0) or right (1). You are not allowed to answer 
                                                anything else besides the number 1 or 0."""},
        {"role": "input", "content": speech},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=100,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.01,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][len(prompt):]



In [ ]:
for index, row in tokenized_test.iterrows():
    print(classification_pipeline(row['text_en']))